# Imports

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import math
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns

from numpy import array, linspace
from sklearn.neighbors import KernelDensity
from matplotlib.pyplot import plot
from scipy.signal import argrelextrema

pd.set_option('display.max_columns', None)
plt.rcParams["figure.figsize"] = (60, 40)
%matplotlib inline

In [2]:
from handlers.DataHandler import DataHandler
# from handlers.ClusterHandler import ClusterHandler

In [3]:
### GLOBAL VARIABLES ###
y_horizon = 13

# Load Data

In [56]:
import numpy as np
import pandas as pd
import math
from sklearn.neighbors import KernelDensity
from scipy.signal import argrelextrema
import matplotlib.pyplot as plt


class ClusterHandler:
    def __init__(self, train_X, train_y, test_X, test_y, eps=0.0005, mergeCluster=True, pltKDE=False, pltNo=0, splitLargest=True):
        self.train_X = train_X
        self.train_y = train_y
        self.test_X = test_X
        self.test_y = test_y
        self.eps = eps
        self.mergeCluster = mergeCluster
        self.pltKDE = pltKDE
        self.pltNo = pltNo
        self.splitLargest = splitLargest

        # Initialize dictionaries to store cluster statistics
        self.feature_cluster_stats = {}
        self.y_cluster_stats = {}

    def cluster(self):
        """Main clustering method that processes all columns."""
        train_X_res = []
        train_y_res = []
        test_X_res = []
        test_y_res = []

        assert self.train_X.shape[1] == self.test_X.shape[1], "Train and test data must have the same number of columns"
        assert self.train_y.shape[1] == self.test_y.shape[1], "Train_y and test_y data must have the same number of columns"
        assert self.train_X.columns.tolist() == self.test_X.columns.tolist(), "Train and test data must have the same columns"
        assert self.train_y.columns.tolist() == self.test_y.columns.tolist(), "Train_y and test_y data must have the same columns"

        # Cluster each feature column
        for column in self.train_X.columns:
            temp, test_temp = self._process_column(column, is_target=False)
            train_X_res.append(temp)
            test_X_res.append(test_temp)

        # Combine clustered results for features
        train_X_res = pd.concat(train_X_res, axis=1)
        test_X_res = pd.concat(test_X_res, axis=1)

        # Cluster each target variable
        for column in self.train_y.columns:
            temp, test_temp = self._process_column(column, is_target=True)
            train_y_res.append(temp)
            test_y_res.append(test_temp)

        # Combine clustered results for target variables
        train_y_res = pd.concat(train_y_res, axis=1)
        test_y_res = pd.concat(test_y_res, axis=1)
        
        return train_X_res, train_y_res, test_X_res, test_y_res

    def _process_column(self, column_name, is_target=False):
        """Process a single column for clustering."""
        temp, testTemp, clsDic = self._cluster_column(column_name, is_target)

        # Remove temporary columns
        temp.drop([f"Cluster_{column_name}", "Data", "Cluster_Mean", "Cluster_Std"], axis=1, inplace=True)
        testTemp.drop(["Data"], axis=1, inplace=True)

        if is_target:
            # temp.reset_index(drop=True, inplace=True)
            temp.index = self.train_y.index
            testTemp.index = self.test_y.index
        else:
            # testTemp.reset_index(drop=True, inplace=True)
            testTemp.index = self.test_X.index
            temp.index = self.train_X.index

        # Append results
        # toTrain.append(temp)
        # toTest.append(testTemp)

        # Store cluster statistics
        if is_target:
            self.y_cluster_stats[column_name] = clsDic
        else:
            self.feature_cluster_stats[column_name] = clsDic
        return temp, testTemp

    def _cluster_column(self, column_name, is_target=False):
        """Cluster a single column of data."""
        # Extract data
        spl = (self.train_y if is_target else self.train_X)[column_name].to_numpy().reshape(-1, 1)
        test_spl = (self.test_y if is_target else self.test_X)[column_name].to_numpy()

        # Perform KDE and find splits
        splits = self._kde_splits(spl)

        # Assign initial clusters
        cls1 = np.sum(spl.reshape(-1)[:, np.newaxis] >= splits, axis=1)

        # Create temporary DataFrames
        temp = pd.DataFrame({'Data': spl.reshape(-1), f'Cluster_{column_name}': cls1})
        testTemp = pd.DataFrame({'Data': test_spl})

        # Calculate initial cluster statistics
        self._calculate_cluster_stats(temp, f'Cluster_{column_name}')

        # Merge small clusters if enabled
        if self.mergeCluster:
            self._merge_clusters(temp, f'Cluster_{column_name}')

        # Calculate membership functions
        clsDic = self._calculate_memberships(temp, testTemp, f'Cluster_{column_name}', column_name, is_target)

        print(f"{column_name} had {len(clsDic)} clusters")
        return temp, testTemp, clsDic

    def _kde_splits(self, spl):
        """Perform KDE and find splits in the data."""
        kde = KernelDensity(kernel='gaussian', bandwidth=self.eps).fit(spl)
        s = np.linspace(min(spl), max(spl))
        e = kde.score_samples(s.reshape(-1, 1))
        mi = argrelextrema(e, np.less)[0]
        splits = s[mi].reshape(-1)

        if self.splitLargest:
            splits = self._split_largest_cluster(splits)

        return splits

    def _split_largest_cluster(self, splits):
        """Split the largest cluster."""
        differences = np.abs(np.diff(splits))
        max_diff_index = np.argmax(differences)
        middle_element = (splits[max_diff_index] + splits[max_diff_index + 1]) / 2
        return np.insert(splits, max_diff_index + 1, middle_element)

    def _calculate_cluster_stats(self, df, cluster_col):
        """Calculate mean and std for each cluster."""
        for cluster_id, cluster_data in df.groupby(cluster_col):
            df.loc[df[cluster_col] == cluster_id, 'Cluster_Mean'] = cluster_data['Data'].mean()
            df.loc[df[cluster_col] == cluster_id, 'Cluster_Std'] = cluster_data['Data'].std(ddof=1)

    def _merge_clusters(self, df, cluster_col):
        """Merge small clusters."""
        clusterNo = sorted(df[cluster_col].unique())
        i, j = 0, 1
        while i < len(clusterNo) and j < len(clusterNo):
            lCls, rCls = clusterNo[i], clusterNo[j]
            if len(df[df[cluster_col] == lCls]) < 5 or len(df[df[cluster_col] == rCls]) < 5:
                df.loc[df[cluster_col] == rCls, cluster_col] = lCls
                newMean = df[df[cluster_col] == lCls]['Data'].mean()
                newStd = df[df[cluster_col] == lCls]['Data'].std()
                df.loc[df[cluster_col] == lCls, 'Cluster_Mean'] = newMean
                df.loc[df[cluster_col] == lCls, 'Cluster_Std'] = newStd
                clusterNo.pop(j)
            else:
                i += 1
                j += 1

    def _calculate_memberships(self, temp, testTemp, cluster_col, column_name, is_target):
        """Calculate membership functions for each cluster."""
        clsDic = {}
        for clsNo, cluster_data in enumerate(temp.groupby(cluster_col)):
            _, data = cluster_data
            sampleMean = data['Data'].mean()
            sampleDevi = data['Data'].std(ddof=1)

            # Handle prefix assignment
            prefix = 'y' if is_target else column_name
            pdf_col = f'PDF_{prefix}_{clsNo}'

            temp[pdf_col] = temp["Data"].apply(lambda x: self._membership(x, sampleMean, sampleDevi))
            testTemp[pdf_col] = testTemp["Data"].apply(lambda x: self._membership(x, sampleMean, sampleDevi))

            clsDic[clsNo] = {'mean': sampleMean, 'std': sampleDevi}

        return clsDic

    def _membership(self, x, sampleMean, sampleDevi):
        """Calculate Gaussian membership."""
        return math.exp(-((x - sampleMean)**2 / (2 * (sampleDevi**2))))

    def clusterGraph(self):
        """Generate cluster distribution graphs."""
        fig, axs = plt.subplots(12, 3, figsize=(45,55))  # 1 row, 2 columns
        for z in range(0, 17): # 16 time steps, ignore t
            a = np.where(self.means[z] == 0)[0]
            if len(a)==0:
                a = len(self.means[z])
            else:
                a = a[0]
            for i in range(a):
                axs[z//3][z%3].scatter(self.train["Close_t-{}".format(z)], self.train["PDF_{}_{}".format(z, i)])
            axs[z//3][z%3].set_xlabel('Closing Value at t-{}'.format(z), fontsize = 20)
            axs[z//3][z%3].set_ylabel('Membership Value', fontsize = 20)
            axs[z//3][z%3].set_title('Cluster Distribution for t-{}'.format(z), fontsize = 30)
            
        for z in range(0, 16): # 15 time steps, ignore t
            a = np.where(self.meansMomentum[z] == 0)[0]
            if len(a)==0:
                a = len(self.meansMomentum[z])
            else:
                a = a[0]
            for i in range(a):
                axs[(z+16)//3][(z+16)%3].scatter(self.train["Close_t-{}.2".format(z)], self.train["PDF_{}.2_{}".format(z, i)])
            axs[(z+16)//3][(z+16)%3].set_xlabel('Closing Value at t-{}.2'.format(z), fontsize = 20)
            axs[(z+16)//3][(z+16)%3].set_ylabel('Membership Value', fontsize = 20)
            axs[(z+16)//3][(z+16)%3].set_title('Cluster Distribution for t-{}.2'.format(z), fontsize = 30)

        for i in range(len(self.y_cluster_stats)):
                axs[-1][-1].scatter(self.train["Close_t+{}".format(self.yTarget)], self.train["PDF_y_{}".format(i)])
        axs[-1][-1].set_xlabel('Closing Value at t+{}'.format(self.yTarget), fontsize = 20)
        axs[-1][-1].set_ylabel('Membership Value', fontsize = 20)
        axs[-1][-1].set_title('Cluster Distribution for t-{}'.format(self.yTarget), fontsize = 30)
        # plt.tight_layout()    
        plt.show()

In [57]:
# Get the data
train_X, train_y, test_X, test_y = DataHandler.getData("MS", "1998-01-01", "2015-12-31", "2016-01-01", "2023-01-01", y_horizon)

# Create and use the Cluster object
cls = ClusterHandler(train_X, train_y, test_X, test_y, eps=0.00001, mergeCluster=True, splitLargest=True)
clustered_train_X, clustered_train_y, clustered_test_X, clustered_test_y = cls.cluster()

# Access the clustering results
feature_cluster_stats = cls.feature_cluster_stats
y_cluster_stats = cls.y_cluster_stats

Open had 16 clusters
High had 15 clusters
Low had 14 clusters
Close had 19 clusters
Volume had 4 clusters
daily_log_return_0 had 8 clusters
daily_log_return_1 had 8 clusters
daily_log_return_2 had 8 clusters
daily_log_return_3 had 8 clusters
daily_log_return_4 had 8 clusters
daily_log_return_5 had 8 clusters
daily_log_return_6 had 8 clusters
daily_log_return_7 had 8 clusters
daily_log_return_8 had 8 clusters
daily_log_return_9 had 8 clusters
daily_log_return_10 had 8 clusters
daily_log_return_11 had 8 clusters
daily_log_return_12 had 8 clusters
intraday_log_return_0 had 9 clusters
intraday_log_return_1 had 9 clusters
intraday_log_return_2 had 9 clusters
intraday_log_return_3 had 9 clusters
intraday_log_return_4 had 9 clusters
intraday_log_return_5 had 9 clusters
intraday_log_return_6 had 9 clusters
intraday_log_return_7 had 9 clusters
intraday_log_return_8 had 9 clusters
intraday_log_return_9 had 9 clusters
intraday_log_return_10 had 9 clusters
intraday_log_return_11 had 9 clusters
int

/var/folders/ng/6yknzn1x345dnygqqxr3qjw00000gn/T/ipykernel_69952/2222618881.py:180: RuntimeWarning: divide by zero encountered in scalar divide
  return math.exp(-((x - sampleMean)**2 / (2 * (sampleDevi**2))))
/var/folders/ng/6yknzn1x345dnygqqxr3qjw00000gn/T/ipykernel_69952/2222618881.py:180: RuntimeWarning: invalid value encountered in scalar divide
  return math.exp(-((x - sampleMean)**2 / (2 * (sampleDevi**2))))
/var/folders/ng/6yknzn1x345dnygqqxr3qjw00000gn/T/ipykernel_69952/2222618881.py:180: RuntimeWarning: divide by zero encountered in scalar divide
  return math.exp(-((x - sampleMean)**2 / (2 * (sampleDevi**2))))
/var/folders/ng/6yknzn1x345dnygqqxr3qjw00000gn/T/ipykernel_69952/2222618881.py:180: RuntimeWarning: invalid value encountered in scalar divide
  return math.exp(-((x - sampleMean)**2 / (2 * (sampleDevi**2))))
/var/folders/ng/6yknzn1x345dnygqqxr3qjw00000gn/T/ipykernel_69952/2222618881.py:180: RuntimeWarning: divide by zero encountered in scalar divide
  return math.exp(

1m_mom had 9 clusters
3m_mom had 15 clusters
6m_mom had 15 clusters
12m_mom had 16 clusters
18m_mom had 16 clusters
mom_change_1m_3m had 11 clusters
mom_change_3m_6m had 16 clusters
mom_change_6m_12m had 15 clusters
y_log_return_0 had 8 clusters
y_log_return_1 had 4 clusters
y_log_return_2 had 10 clusters
y_log_return_3 had 8 clusters
y_log_return_4 had 5 clusters
y_log_return_5 had 9 clusters
y_log_return_6 had 8 clusters
y_log_return_7 had 8 clusters
y_log_return_8 had 11 clusters
y_log_return_9 had 9 clusters
y_log_return_10 had 10 clusters
y_log_return_11 had 13 clusters
y_log_return_12 had 10 clusters


# Functions and model definition

In [82]:
import torch
import torch.nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import math
from sklearn.metrics import r2_score
from torch.nn import TransformerEncoder, TransformerEncoderLayer

In [83]:
class TransformerModel(torch.nn.Module):
    def __init__(self, input_dim, output_dim, num_heads=10, ff_dim=32, num_transformer_blocks=4, mlp_units=256, dropout=0.25, noHiddenLayers = 1, sigmoidOrSoftmax = 0):
        super(TransformerModel, self).__init__()

        # Encoder layer
        encoder_layers = TransformerEncoderLayer(d_model=input_dim, nhead=num_heads, dim_feedforward=mlp_units)
        self.transformer_encoder = TransformerEncoder(encoder_layers, num_layers=num_transformer_blocks)
        
        fcList = [torch.nn.Linear(input_dim, mlp_units), torch.nn.ReLU()]
        for i in range(noHiddenLayers):
            fcList.append(torch.nn.Linear(mlp_units, mlp_units//2))
            fcList.append(torch.nn.ReLU())
            mlp_units = mlp_units//2
        fcList.append(torch.nn.Linear(mlp_units, output_dim))
        if sigmoidOrSoftmax == 0:
            fcList.append(torch.nn.Sigmoid())
        else:
            fcList.append(torch.nn.Softmax())
        
        self.fc = torch.nn.Sequential(*fcList)

    def forward(self, src):
        # src shape: (seq_length, batch_size, input_dim)
        
        # Pass input through the transformer encoder
        encoder_output = self.transformer_encoder(src)
        encoder_output = encoder_output[:,:]
        output = self.fc(encoder_output)
        return output    
    
def increaseInstancesExtreme(train, thresholdToIncrease=0.03):
    extraData = train[(train[f"Close_t+{yTarget}"]>thresholdToIncrease) | (train[f"Close_t+{yTarget}"]<-thresholdToIncrease)]
    return pd.concat([train, extraData] ,axis = 0)

def weightAvg(pred, noClusters, deltaOnly = True):
    center = [i[0] for i in cls.yClsDic.values()]
    std = [i[1] for i in cls.yClsDic.values()]
    std = np.tile(center,(pred.shape[0],1))
    centers = np.tile(center, (pred.shape[0], 1))
    pred = np.nan_to_num(pred, nan=0, posinf=0, neginf=0)
    denominator = pred
    denominator = denominator.sum(axis=1, keepdims=True)
    numerator = (pred * centers)
    numerator = numerator.sum(axis = 1, keepdims = True)
    result = numerator / denominator
    result = np.squeeze(result)
    if not deltaOnly:
        return (result * cls.test.Close.to_numpy())+cls.test.Close.to_numpy()
    else:
        return (result * cls.test.Close.to_numpy())
    
def padData(X, X_test, padDim):
    zeros_array = np.zeros((X.shape[0], padDim))
    zeros_array_test = np.zeros((X_test.shape[0], padDim))

    X = np.concatenate((X, zeros_array), axis=1)
    X_test = np.concatenate((X_test, zeros_array_test), axis=1)
    return X, X_test

def testData(pred, Y_test):
    res = (pred-Y_test)**2
    res[np.isnan(res)]=0
    mse = np.mean(res)
    return mse

# Train

In [84]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from sklearn.metrics import r2_score
import numpy as np
import math

def train_ga_model(num_heads, feed_forward_dim, num_transformer_blocks, mlp_units, dropout_rate, learning_rate, num_mlp_layers, num_epochs, activation_function):
    # Detect GPU availability
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Training on {device}")

    # Load and pad data
    X, Y, X_test, Y_test = clustered_train_X.to_numpy(), clustered_train_y.to_numpy(), clustered_test_X.to_numpy(), clustered_test_y.to_numpy()
    x_padded, x_test_padded = padData(X, X_test, math.ceil(X.shape[1] / num_heads) * num_heads - X.shape[1])

    # Initialize the model
    model = TransformerModel(
        input_dim=x_padded.shape[1], 
        output_dim=Y.shape[1],
        num_heads=num_heads, 
        ff_dim=feed_forward_dim, 
        num_transformer_blocks=num_transformer_blocks, 
        mlp_units=mlp_units, 
        dropout=dropout_rate, 
        sigmoidOrSoftmax=activation_function
    ).double()

    # Send model to the detected device
    model = model.to(device)

    # Set loss function and optimizer
    criterion = torch.nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader for training
    assert x_padded.shape[0] == Y.shape[0]
    train_dataset = TensorDataset(torch.from_numpy(x_padded), torch.from_numpy(Y))
    train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)

    # Training loop
    for epoch in range(num_epochs):
        epoch_loss = 0
        model.train()

        for inputs, targets in train_dataloader:
            # Move data to the correct device
            inputs, targets = inputs.to(device), targets.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            epoch_loss += loss.item()

            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(train_dataloader):.4f}', end=" ")

        # Evaluate on test data
        test_dataset = TensorDataset(torch.from_numpy(x_test_padded), torch.from_numpy(Y_test))
        test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)

        model.eval()
        pred_list = []
        with torch.no_grad():
            for inputs, _ in test_dataloader:
                pred = model(inputs.to(device)).cpu()
                pred_list.append(pred)

        mse = testData(np.concatenate(pred_list), Y_test)
        # print(f'Test Data MSE: {mse:.7f}')

        # Calculate R2 score
        pred = np.concatenate(pred_list)
        pred[np.isnan(pred)] = 0
        res = weightAvg(pred, len(cls.yClsDic), isTestY=True)
        res = np.nan_to_num(res, nan=0, posinf=0, neginf=0)
        pred_closing = addResToClosing(cls, res)[:-yTarget]
        actual_closing = cls.test.Close[yTarget:].to_numpy()
        r2_score_value = r2_score(actual_closing, pred_closing)
        print(" R2 Score:", r2_score_value)

    return model, r2_score_value, pred

In [85]:
x_padded, x_test_padded = train_ga_model(
    num_heads=1, 
    feed_forward_dim=1, 
    num_transformer_blocks=3, 
    mlp_units=64, 
    dropout_rate=0.1, 
    learning_rate=5e-06, 
    num_mlp_layers=3, 
    num_epochs=50, 
    activation_function=0  # 0 for sigmoid, 1 for softmax
)

Training on cpu


/Users/sam/anaconda3/envs/dsenv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [96]:
(x_padded == clustered_train_X.to_numpy())

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [88]:
x_test_padded.shape

(1372, 590)

In [91]:
clustered_train_X.to_numpy().shape

(4138, 590)

In [92]:
clustered_test_X.to_numpy().shape

(1372, 590)

In [ ]:
model, r2, pred = train_ga_model(
    num_heads=1, 
    feed_forward_dim=1, 
    num_transformer_blocks=3, 
    mlp_units=64, 
    dropout_rate=0.1, 
    learning_rate=5e-06, 
    num_mlp_layers=3, 
    num_epochs=50, 
    activation_function=0  # 0 for sigmoid, 1 for softmax
)
print(r2)

In [75]:
from sklearn.metrics import r2_score

r2_score(train_X['Close'].iloc[13:], train_X['Close'].shift(13).iloc[13:])

0.935833218015573

In [67]:
train

NameError: name 'train' is not defined

In [70]:
train_X['Close']

Date
1999-07-06    25.664207
1999-07-07    25.695587
1999-07-08    25.774012
1999-07-09    25.993652
1999-07-12    25.601460
                ...    
2015-12-07    27.070307
2015-12-08    26.586359
2015-12-09    26.047758
2015-12-10    26.086788
2015-12-11    25.040825
Name: Close, Length: 4138, dtype: float64

In [72]:
train_X['Close'].shift(1)

Date
1999-07-06          NaN
1999-07-07    25.664207
1999-07-08    25.695587
1999-07-09    25.774012
1999-07-12    25.993652
                ...    
2015-12-07    27.569885
2015-12-08    27.070307
2015-12-09    26.586359
2015-12-10    26.047758
2015-12-11    26.086788
Name: Close, Length: 4138, dtype: float64